## E. coli GEM-PRO for all proteins using UniProt proteome FASTA file

- Also includes SWISS-MODELs

In [20]:
import sys
import logging
import os.path as op
import pandas as pd
from ssbio.pipeline.gempro import GEMPRO
import ssbio.protein.sequence.utils.fasta

# Logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] %(levelname)s: %(message)s', datefmt="%Y-%m-%d %H:%M")
handler.setFormatter(formatter)
logger.handlers = [handler]

# Printing multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Use the UniProt E. coli FASTA file

In [3]:
FASTA_FILE = './171205-uniprot_ecoli_proteome.fasta'
ss = ssbio.protein.sequence.utils.fasta.load_fasta_file(FASTA_FILE)

In [4]:
len(ss)

4306

#### Convert UniProt IDs to bnumbers

In [5]:
uniprot_to_bnumber = {}

for i, r in pd.read_csv('./171205-uniprot_ecoli_mapping_to_bnum_extra1.tab', sep='\t')[['Entry', 'Gene names  (ordered locus )']].iterrows():
    bnum = r['Gene names  (ordered locus )'].split(' ')[0]
    if bnum.startswith('b'):
        uniprot_to_bnumber[r['Entry']] = bnum
    
for i, r in pd.read_csv('./171205-uniprot_ecoli_mapping_to_bnum_extra2.tab', sep='\t')[['Entry', 'Gene names  (ordered locus )']].iterrows():
    if r['Entry'] in uniprot_to_bnumber:
        print(r['Entry'])
    bnum = r['Gene names  (ordered locus )'].split(' ')[0]
    if bnum.startswith('b'):
        uniprot_to_bnumber[r['Entry']] = bnum
    
for i, r in pd.read_csv('./171205-uniprot_ecoli_mapping_to_bnum_extra3.tab', sep='\t')[['Entry', 'Gene names  (ordered locus )']].iterrows():
    if r['Entry'] in uniprot_to_bnumber:
        print(r['Entry'])
    try:
        bnum = r['Gene names  (ordered locus )'].split(' ')[0]
        if bnum.startswith('b'):
            uniprot_to_bnumber[r['Entry']] = bnum
    except:
        continue

In [6]:
from bioservices import UniProt
bs_unip = UniProt()
uniprot_to_kegg = bs_unip.mapping(fr='ACC', to='KEGG_ID', query=[x.id.split('|')[1] for x in ss])

for k,v in uniprot_to_kegg.items():
    for x in v:
        if 'eco' in x:
            if k in uniprot_to_bnumber:
                continue
            else:
                uniprot_to_bnumber[k] = x.split(':')[1]

#### Convert FASTA file to dict of seqs with the bnumber

In [7]:
sss = ssbio.protein.sequence.utils.fasta.load_fasta_file_as_dict_of_seqs(FASTA_FILE)

In [8]:
final = {}

for k,v in sss.items():
    unip = k.split('|')[1]
    if unip not in uniprot_to_bnumber:
        print(unip)
    else:
        final[uniprot_to_bnumber[unip]] = v

#### Make GEM-PRO

In [9]:
GEM_NAME = 'ecoli_gp'
ROOT_DIR = '/home/nathan/projects_unsynced/'

In [10]:
# Create the GEM-PRO project
my_gempro = GEMPRO(gem_name=GEM_NAME, root_dir=ROOT_DIR, genes_and_sequences=final, pdb_file_type='mmtf')

In [11]:
# Remap back to uniprot
bnumber_to_uniprot = {v:k for k,v in uniprot_to_bnumber.items()}
my_gempro.manual_uniprot_mapping(bnumber_to_uniprot)
my_gempro.set_representative_sequence()

HBox(children=(IntProgress(value=0, max=4305), HTML(value='')))

HBox(children=(IntProgress(value=0, max=4305), HTML(value='')))

In [12]:
my_gempro.map_uniprot_to_pdb()
my_gempro.blast_seqs_to_pdb(all_genes=True, seq_ident_cutoff=.75, evalue=0.00001)

HBox(children=(IntProgress(value=0, max=4305), HTML(value='')))

HBox(children=(IntProgress(value=0, max=4305), HTML(value='')))

In [13]:
import shutil

In [14]:
# Downloading and loading SWISS-MODEL models
from tqdm import tqdm_notebook as tqdm

import json
with open('/home/nathan/projects_archive/homology_models/ECOLI/SWISS-MODEL_Repository/INDEX_JSON') as f:
    loaded = json.load(f)
    
from collections import defaultdict
uniprot_to_swissmodel = defaultdict(list)

for x in loaded:
    uni = x['uniprot_ac']
    file_path = op.join('/home/nathan/projects_archive/homology_models/ECOLI/SWISS-MODEL_Repository/', 
                        uni[:2], uni[2:4], uni[4:], 'swissmodel', 
                        '{}_{}_{}_{}.pdb'.format(x['from'], x['to'], x['template'], x['coordinate_id']))
    if op.exists(file_path):
        uniprot_to_swissmodel[uni].append(file_path)
        
        
# Load models into gempro
for g in tqdm(my_gempro.genes_with_a_representative_sequence):
    if g.protein.representative_sequence.uniprot in uniprot_to_swissmodel:
        for m in uniprot_to_swissmodel[g.protein.representative_sequence.uniprot]:
            
            ident = '{}_{}'.format(g.id, op.basename(m).split('.')[0])
            
            # Copy and rename files...
            new_folder = '/home/nathan/projects_archive/homology_models/ECOLI/SWISS-MODEL_renamed'
            renamer = '{}.pdb'.format(ident)
            new_file = op.join(new_folder, renamer)
            _ = shutil.copy(m, new_file)
            
            new_homology = g.protein.load_pdb(pdb_id=ident, pdb_file=new_file, file_type='pdb', is_experimental=False)

HBox(children=(IntProgress(value=0, max=4305), HTML(value='')))

In [ ]:
my_gempro.set_representative_structure(allow_missing_on_termini=.6, allow_deletions=True, seq_ident_cutoff=0.3)

HBox(children=(IntProgress(value=0, max=4305), HTML(value='')))

ERROR:ssbio.core.protein:b4252_1_150_1s4c-_: alignment failed to run, unable to check structure
ERROR:ssbio.protein.structure.utils.structureio:/home/nathan/projects_unsynced/ecoli_gp/genes/b3303/b3303_protein/structures/4v9o.mmtf: unable to save structure in PDB file format


%c requires int or char


ERROR:root:Unknown error with PDB ID 4v9o
Traceback (most recent call last):
  File "/mnt/projects/ssbio/ssbio/core/protein.py", line 1675, in set_representative_structure
    force_rerun=force_rerun)
  File "/mnt/projects/ssbio/ssbio/core/protein.py", line 1506, in _representative_structure_setter
    self.representative_structure.parse_structure()
  File "/mnt/projects/ssbio/ssbio/protein/structure/structprop.py", line 138, in parse_structure
    if not self.structure_path:
  File "/mnt/projects/ssbio/ssbio/protein/structure/structprop.py", line 105, in structure_path
    raise OSError('{}: structure file not available'.format(self.id))
OSError: REP-4v9o: structure file not available
ERROR:ssbio.protein.structure.utils.structureio:/home/nathan/projects_unsynced/ecoli_gp/genes/b3303/b3303_protein/structures/4v9p.mmtf: unable to save structure in PDB file format


%c requires int or char


...

ERROR:root:Unknown error with PDB ID 4v9p
Traceback (most recent call last):
  File "/mnt/projects/ssbio/ssbio/core/protein.py", line 1675, in set_representative_structure
    force_rerun=force_rerun)
  File "/mnt/projects/ssbio/ssbio/core/protein.py", line 1506, in _representative_structure_setter
    self.representative_structure.parse_structure()
  File "/mnt/projects/ssbio/ssbio/protein/structure/structprop.py", line 138, in parse_structure
    if not self.structure_path:
  File "/mnt/projects/ssbio/ssbio/protein/structure/structprop.py", line 105, in structure_path
    raise OSError('{}: structure file not available'.format(self.id))
OSError: REP-4v9p: structure file not available
ERROR:ssbio.protein.structure.utils.structureio:/home/nathan/projects_unsynced/ecoli_gp/genes/b0191/b0191_protein/structures/4v95.mmtf: unable to save structure in PDB file format


%c requires int or char


ERROR:root:Unknown error with PDB ID 4v95
Traceback (most recent call last):
  File "/mnt/projects/ssbio/ssbio/core/protein.py", line 1675, in set_representative_structure
    force_rerun=force_rerun)
  File "/mnt/projects/ssbio/ssbio/core/protein.py", line 1506, in _representative_structure_setter
    self.representative_structure.parse_structure()
  File "/mnt/projects/ssbio/ssbio/protein/structure/structprop.py", line 138, in parse_structure
    if not self.structure_path:
  File "/mnt/projects/ssbio/ssbio/protein/structure/structprop.py", line 105, in structure_path
    raise OSError('{}: structure file not available'.format(self.id))
OSError: REP-4v95: structure file not available
ERROR:ssbio.core.protein:b1413_68_526_5aor-_: alignment failed to run, unable to check structure
ERROR:ssbio.core.protein:b1222_382_585_3gig-_: alignment failed to run, unable to check structure
ERROR:ssbio.core.protein:P0AFA2_b1222_382_585_3gig-_: structure alignment not found, please run the alignment 

%c requires int or char


ERROR:root:Unknown error with PDB ID 4v9o
Traceback (most recent call last):
  File "/mnt/projects/ssbio/ssbio/core/protein.py", line 1675, in set_representative_structure
    force_rerun=force_rerun)
  File "/mnt/projects/ssbio/ssbio/core/protein.py", line 1506, in _representative_structure_setter
    self.representative_structure.parse_structure()
  File "/mnt/projects/ssbio/ssbio/protein/structure/structprop.py", line 138, in parse_structure
    if not self.structure_path:
  File "/mnt/projects/ssbio/ssbio/protein/structure/structprop.py", line 105, in structure_path
    raise OSError('{}: structure file not available'.format(self.id))
OSError: REP-4v9o: structure file not available
ERROR:ssbio.core.protein:b2920_8_490_2nvv-_: alignment failed to run, unable to check structure
ERROR:ssbio.core.protein:b0417_5_323_5dd7-_: alignment failed to run, unable to check structure
ERROR:ssbio.core.protein:b3212_12_1473_2vdc-_: alignment failed to run, unable to check structure
ERROR:ssbio.cor

In [ ]:
my_gempro.save_json('./171212-ecoli_whole_genome_swissmodel_only_lower_cutoffs_gp.json')
my_gempro.save_pickle('./171212-ecoli_whole_genome_swissmodel_only_lower_cutoffs_gp.pckl')

## Loading and sandbox

In [1]:
import ssbio.io

In [2]:
my_gempro = ssbio.io.load_pickle('./171212-ecoli_whole_genome_swissmodel_only_lower_cutoffs_gp.pckl')

In [4]:
# b3343(2) AND b3344(2) AND b3345(2)
b3343 = my_gempro.genes.get_by_id('b3343')
b3344 = my_gempro.genes.get_by_id('b3344')
b3345 = my_gempro.genes.get_by_id('b3345')

In [18]:
b3343.protein.structures

[<PDBProp 2d1p at 0x7eff4a480160>,
 <StructProp b3343_1_95_2d1p at 0x7eff4a480400>,
 <StructProp REP-2d1p at 0x7eff4a480438>]

In [10]:
b3344.protein.structures

[<PDBProp 2d1p at 0x7eff4ed02dd8>,
 <StructProp b3344_1_119_2d1p at 0x7eff4ed170b8>,
 <StructProp REP-2d1p at 0x7eff4ed170f0>]

In [11]:
b3345.protein.structures

[<PDBProp 2d1p at 0x7eff4a7b9080>,
 <StructProp b3345_1_128_2d1p at 0x7eff4a7b9320>,
 <StructProp REP-2d1p at 0x7eff4a7b9358>]

In [21]:
b3343.protein.representative_chain
b3344.protein.representative_chain
b3345.protein.representative_chain

'C'

'B'

'A'

In [25]:
b3343.protein.structures[0].mapped_chains
b3344.protein.structures[0].mapped_chains
b3345.protein.structures[0].mapped_chains

['C', 'F', 'I']

['B', 'E', 'H']

['A', 'D', 'G']